In [1]:
import streamlit as st
from PyPDF2 import PdfReader
from langchain.text_splitter import RecursiveCharacterTextSplitter
import os
from langchain_google_genai import GoogleGenerativeAIEmbeddings
import google.generativeai as genai
from langchain.vectorstores import FAISS
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.chains.question_answering import load_qa_chain
from langchain.prompts import PromptTemplate
from dotenv import load_dotenv


c:\Users\vidit\.conda\envs\tutor\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
import os
print(os.getcwd())

In [2]:
from langchain.document_loaders import TextLoader

loader = TextLoader(r"C:/vidit/codes/PROJECTS/Tutor-app/backend/knowledge_store/transformer_research_paper.pdf")
data = loader.load()
print(data[0].page_content[0:2500])

RuntimeError: Error loading C:/vidit/codes/PROJECTS/Tutor-app/backend/knowledge_store/transformer_research_paper.pdf

In [24]:
from langchain.document_loaders.csv_loader import CSVLoader

loader = CSVLoader("C:/vidit/codes/PROJECTS/Tutor-app/artifacts/movies.csv")
data = loader.load()
print(data[1].page_content)

movie_id: 102
title: Doctor Strange in the Multiverse of Madness
industry: Hollywood
release_year: 2022
imdb_rating: 7
studio: Marvel Studios
language_id: 5
budget: 200
revenue: 954.8
unit: Millions
currency: USD


In [41]:
from PyPDF2 import PdfReader

text=""
pdf_docs = [r'C:/vidit/codes/PROJECTS/Tutor-app/artifacts/ssrn-4878177.pdf']
for pdf in pdf_docs:
    pdf_reader= PdfReader(pdf)
    for page in pdf_reader.pages:
        text+= page.extract_text()
print(text)

Abstract
The study offers a comprehensive analysis of global 
research on Chatbot and ChatGPT from 2002 to 2023. 
A rapid research growth has been noted after the 
year 2017. The research growth, authorship analysis, 
keyword analysis, citation pattern, co-occurrence 
of keywords and thematic analysis were analysed. 
The USA leads in publications and citations, while 
the Norway achieved the highest average citations. 
Keyword trends highlighted “Chatbot,” “Artificial 
intelligence” and “ChatGPT” as research domains. 
Thematic clusters emerged from co-occurrence 
analysis. Finally, this study maps the dynamic 
evolution of Chatbot and ChatGPT research in the first 
two decades of the 21st century and offers insights for 
researchers, practitioners and policymakers in the AI 
research field. 
Keywords:  Bibliometrics, Chabot, ChatGPT, Lotka 
Law, Artificial Intelligence, Cluster AnalysisGlobal Research Trends on Chatbot and ChatGPT 
in the 21st Century
Kunjan Prasad Gupta*, Manendra Kum

In [51]:
from langchain.document_loaders import UnstructuredURLLoader
loader = UnstructuredURLLoader(
    urls = [
        "https://www.moneycontrol.com/news/business/banks/hdfc-bank-re-appoints-sanmoy-chakrabarti-as-chief-risk-officer-11259771.html",
        "https://www.moneycontrol.com/news/business/markets/market-corrects-post-rbi-ups-inflation-forecast-icrr-bet-on-these-top-10-rate-sensitive-stocks-ideas-11142611.html"
    ]
)

data = loader.load()
text = data[1].page_content
text

'English\n\nHindi\n\nGujarati\n\nSpecials\n\nMoneycontrol Trending Stock\n\nInfosys\xa0INE009A01021, INFY, 500209\n\nState Bank of India\xa0INE062A01020, SBIN, 500112\n\nYes Bank\xa0INE528G01027, YESBANK, 532648\n\nBank Nifty\n\nNifty 500\n\nQuotes\n\nMutual Funds\n\nCommodities\n\nFutures & Options\n\nCurrency\n\nNews\n\nCryptocurrency\n\nForum\n\nNotices\n\nVideos\n\nGlossary\n\nAll\n\nHello, Login Hello, LoginLog-inor Sign-UpMy AccountMy Profile My PortfolioMy WatchlistFREE Credit Score₹100 Cash RewardMy AlertsMy MessagesPrice AlertsMy Profile My PROMy PortfolioMy WatchlistFREE Credit Score₹100 Cash RewardMy AlertsMy MessagesPrice AlertsLogoutChat with UsDownload AppFollow us on:\n\nGo Ad-Free\n\nMy Alerts\n\nBudget 2        24MarketsHOMEINDIAN INDICESSTOCK ACTIONAll StatsTop GainersTop LosersOnly BuyersOnly Sellers52 Week High52 Week LowPrice ShockersVolume ShockersMost Active StocksGLOBAL MARKETSUS MARKETSSEASONALITY ANALYSISSTOCK SCANNERECONOMIC INDICATORSECONOMIC CALENDARMARKET 

In [77]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

r_splitter = RecursiveCharacterTextSplitter(
    separators = ["\n\n","\n"," "],  # List of separators based on requirement (defaults to ["\n\n", "\n", " "])
    chunk_size = 1000,  
    chunk_overlap  = 100, 
    length_function = len  # Function to calculate size, currently we are using "len" which denotes length of string however you can pass any token counter)
)

chunks = r_splitter.split_text(text)
for chunk in chunks:
    print(len(chunk))

660
928
995
997
257
643
752
769
993
935
942
804
804
883
894
919
733
995
847
808
713
781
996
741
919
992
985
983
987
901


In [85]:
from sentence_transformers import SentenceTransformer

encoder = SentenceTransformer("all-mpnet-base-v2")
vectors = encoder.encode(chunks)

vectors.shape

(30, 768)

In [99]:
# from pinecone import ServerlessSpec

# cloud = os.environ.get('PINECONE_CLOUD') or 'aws'
# region = os.environ.get('PINECONE_REGION') or 'us-east-1'

# spec = ServerlessSpec(cloud=cloud, region=region)

In [111]:
# index_name = 'tutor'
# pc.describe_index(index_name).status['ready']

True

In [112]:
# index = pc.Index(index_name)
# index.describe_index_stats()

{'dimension': 768,
 'index_fullness': 0.0,
 'namespaces': {},
 'total_vector_count': 0}

In [5]:
from langchain_pinecone import PineconeVectorStore
from langchain_community.document_loaders import DirectoryLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain.document_loaders import PyMuPDFLoader
import os
import glob

In [8]:
os.chdir("../")

In [9]:
os.getcwd()

'c:\\vidit\\codes\\PROJECTS\\Tutor-app'

In [10]:
# loader = DirectoryLoader('artifacts', )
loader = PyMuPDFLoader(r'C:/vidit/codes/PROJECTS/Tutor-app/backend/knowledge_store/transformer_research_paper.pdf')

In [11]:
docs = loader.load()

In [18]:
docs = [docs[i].page_content for i in range(len(docs))]

In [19]:
docs

['Provided proper attribution is provided, Google hereby grants permission to\nreproduce the tables and figures in this paper solely for use in journalistic or\nscholarly works.\nAttention Is All You Need\nAshish Vaswani∗\nGoogle Brain\navaswani@google.com\nNoam Shazeer∗\nGoogle Brain\nnoam@google.com\nNiki Parmar∗\nGoogle Research\nnikip@google.com\nJakob Uszkoreit∗\nGoogle Research\nusz@google.com\nLlion Jones∗\nGoogle Research\nllion@google.com\nAidan N. Gomez∗†\nUniversity of Toronto\naidan@cs.toronto.edu\nŁukasz Kaiser∗\nGoogle Brain\nlukaszkaiser@google.com\nIllia Polosukhin∗‡\nillia.polosukhin@gmail.com\nAbstract\nThe dominant sequence transduction models are based on complex recurrent or\nconvolutional neural networks that include an encoder and a decoder. The best\nperforming models also connect the encoder and decoder through an attention\nmechanism. We propose a new simple network architecture, the Transformer,\nbased solely on attention mechanisms, dispensing with recurrenc

In [26]:
index_name = "tutor"

# Split our documents into chunks
text_splitter = RecursiveCharacterTextSplitter(
    separators = ["\n\n","\n"," "], 
    chunk_size = 200,  
    chunk_overlap  = 20, 
    length_function = len 
)
split_docs = text_splitter.split_text(docs[0])



In [27]:
for chunk in split_docs:
    print(len(chunk))

199
198
176
130
153
155
158
153
161
164
114
107
111
112
109
109
111
113
165
172


In [174]:
from pinecone import Pinecone
import dotenv 

api_key = os.environ.get("PINECONE_API_KEY")

pc = Pinecone(api_key=api_key)

index = pc.Index(index_name)
index.describe_index_stats()

{'dimension': 768,
 'index_fullness': 0.0,
 'namespaces': {},
 'total_vector_count': 0}

In [213]:
def store_chunks_in_pinecone(chunks,index,namespace):
    for idx, chunk in enumerate(chunks):
        text = chunk.page_content
        text = text.replace("\n",". ")
        text = text.replace("\t",". ")

        vector = encoder.encode(text)
        vector_id = f"doc_chunk_{idx}"
        
        index.upsert(
            vectors = 
            [
                {
                    "id" : vector_id,
                    "values" : vector,
                    "metadata" : {'source': '','text' : text}
                }
            ] ,
            namespace=namespace,
        )

In [214]:
store_chunks_in_pinecone(docs, index,"test")

VectorDictionaryMissingKeysError: Vector dictionary is missing required fields: ['values']

In [192]:
test_vector = [0.0828915909, -0.0109610604, -0.0137331225, -0.0420022, -0.032611683, -0.0134577537, 0.103452846, 0.0377742872, -0.00628403854, -0.0305299442, -0.0061371671, 0.0326002352, 0.0459518433, 0.0114742322, -0.00552665861, 0.0124429129, 0.0373903438, 0.0424919911, 0.0218762346, 0.00220496743, -0.0544621833, -0.017204307, 0.0353284217, 0.019594159, 0.0374116637, -0.011419653, 0.0510954261, -0.041959811, 0.000983059872, -0.0609145872, 0.0540829152, 0.0160237011, -0.0157552157, 0.0289409012, 0.00000249528011, -0.0324194394, 0.00245248945, 0.0450812727, 0.0368107073, 0.0779069215, 0.0405737422, 0.00738166273, -0.0173286088, -0.01309558, -0.0236868821, -0.0568953864, 0.00794750825, 0.0414108261, -0.0479017571, 0.00768168364, 0.0133750243, 0.00454847841, 0.0618613809, 0.00428978261, 0.0251422245, -0.0187372174, 0.0271532815, 0.039297428, 0.0212325864, 0.0273760725, -0.0178202782, 0.0224938076, -0.0267114155, -0.0221465472, 0.0119918482, 0.0473494716, 0.0185042396, -0.0085211955, 0.0634914, -0.0066223382, 0.130368277, -0.0322167091, 0.0297477879, 0.0405103, -0.0434983522, 0.0461343862, 0.00528288214, 0.0106965285, -0.0366933458, -0.0281769726, 0.0669987574, 0.0634374321, -0.00741368346, 0.0196955092, 0.00154642027, 0.0042987559, -0.0307235, -0.0246864948, 0.0174652375, -0.0502572693, 0.0581222735, -0.0294795334, -0.0107105272, -0.0108148977, 0.0246086381, -0.0348517373, 0.0761027, -0.0309510883, 0.0112329079, -0.0338761918, -0.0438834205, 0.00718821632, 0.028786201, 0.054776378, 0.0407296233, -0.0304144565, -0.0253766868, 0.00251991116, 0.0257006083, 0.0196139105, -0.0337911695, -0.0172141045, 0.0599756725, -0.00442103762, 0.0282742921, -0.0115725799, -0.0976236, -0.0102020465, 0.00874845497, -0.00846910942, -0.00182691205, 0.031970121, -0.0476845205, 0.0217210986, -0.0266993623, 0.021790674, -0.0273364019, 0.0137447491, -0.0218967702, 0.0357382298, 0.0292600282, 0.0125802131, 0.0219685212, -0.0446110331, 0.0116280103, -0.00471792324, -0.0124533288, -0.0141091198, -0.0197460894, 0.0097604366, -0.00561201898, -0.0441992171, 0.0142085021, -0.00680718664, 0.00894029066, 0.01683194, 0.00221669418, -0.0383608378, -0.0198190119, -0.0178433545, -0.0218266491, -0.0308956932, -0.0612981319, -0.00323503488, 0.0534549467, 0.0253105629, 0.0374983102, -0.0391343422, 0.00552831264, -0.0133157214, 0.00770097272, -0.0047106077, 0.030048484, -0.0472838283, -0.0348886959, 0.0164161809, 0.0179464053, 0.014905489, -0.0374043956, -0.0382702313, -0.0117096566, 0.0000637193734, -0.0407352671, 0.065715313, 0.0600267351, -0.00514716, -0.00669236062, 0.0250033084, -0.0202844758, 0.00544328289, 0.0489200577, -0.0530171394, 0.0426089242, 0.0248634741, 0.030221073, 0.00573079893, 0.019111922, -0.00370656815, 0.0119371647, -0.038386941, 0.0111349029, 0.0195080657, -0.0211844575, -0.000547668, 0.041678533, -0.0206953846, 0.0383029, -0.0298785809, -0.00102731318, 0.00113354716, -0.0429773144, 0.0219402853, 0.0485343598, 0.0192676447, 0.00218791049, -0.0154087776, 0.054482352, 0.00936010201, -0.0227399543, 0.000769048929, -0.0650154352, 0.0298100542, 0.0122903334, 0.00514974725, 0.00206985231, 0.0268148668, -0.00893883314, 0.03822143, 0.00157296564, 0.0922141522, -0.00306454208, -0.00289166137, -0.0139496559, -0.0266718846, -0.0318369865, 0.018904414, -0.00300925458, -0.0219890233, 0.0198752768, 0.0342664234, -0.0649806559, -0.0103302663, 0.0309895203, -0.02069778, -0.0298205167, 0.0246445257, 0.0882784575, 0.0927725211, -0.00910731126, -0.0783329681, 0.0386198, -0.0270952154, -0.00508834654, -0.0128751807, 0.0177393053, 0.0222371276, 0.031462796, -0.00596398069, -0.00233020214, -0.038216114, 0.00742547726, 0.0133467689, 0.0558220185, 0.0000948565721, 0.0407240167, 0.0451214574, -0.0706777573, 0.0441380367, -0.0103473477, 0.0200132374, 0.0272670425, -0.0350551829, -0.0249535386, 0.060829334, 0.0208323, -0.0104087433, -0.0337109081, 0.0048388876, -0.0253377501, -0.00130462612, 0.00602822611, 0.000492582447, -0.0214995053, -0.00686765, 0.0423609242, 0.022223508, -0.0213092826, -0.0101239597, -0.0240998846, 0.0583113022, 0.00708251074, 0.039138943, -0.0180068482, 0.000741007738, -0.0151908509, 0.00834727474, 0.0928725898, -0.0786776394, -0.0146090705, -0.0100790346, -0.0197962075, 0.00214834255, -0.0528732, -0.0267982837, -0.032395862, 0.0205077808, -0.0460615754, 0.0331781842, 0.00981911737, 0.03015537, 0.0120492848, -0.0245251693, 0.00464925868, 0.0202805735, -0.00497305393, 0.0479407758, 0.0236330517, -0.048376739, 0.0364764407, -0.0225650724, -0.000130715664, 0.0075394623, 0.0342293642, -0.0228398684, -0.0324904323, -0.0235041492, -0.00646145642, -0.03251715, -0.0203082729, 0.081550993, 0.00153021864, -0.00206605089, -0.0334982648, -0.00809168536, 0.030266894, -0.0391974449, -0.0099335257, -0.0118145011, 0.0240481459, -0.00442961464, 0.0175569393, 0.0457432158, -0.0392796732, -0.0067628338, 0.0264154114, 0.0228378642, -0.0177046936, -0.0636621639, 0.0535767116, -0.0243067518, -0.0520129129, 0.00530843483, 0.031953264, -0.0202128626, 0.0340579711, -0.0334593393, 0.0830619633, -0.0499099605, -0.00812749192, 0.0358885, 0.10766121, -0.0050334828, -0.0213699285, -0.0200720057, -0.00668251142, -0.00839580409, 0.0234798715, -0.0212494228, 0.00975055527, 0.0343365446, -0.0448318683, -0.00179071655, -0.0377485454, 0.0280090868, 0.0397661664, 0.01969992, 0.0642770082, -0.00914323796, -0.081419386, -0.00426997803, 0.0350345634, -0.0289934594, 0.0582522489, 0.0742021278, 0.00769395707, -0.0722953677, -0.0337828733, -0.0300511066, -0.0644483119, 0.00320890103, 0.0134028019, -0.0596864708, 0.092065841, -0.0176240131, -0.0338158831, -0.0538319312, -0.0439913459, -0.0342930034, -0.016865395, 0.0233177748, -0.029398039, -0.0238331966, -0.00499245757, -0.0232033, -0.0461444929, -0.00228560483, -0.0148366624, 0.000474646484, -0.016392041, 0.0102899512, 0.0187038295, -0.0883723199, -0.0166596547, 0.033092171, 0.000605391222, -0.0119482577, 0.0250517931, 0.0497918464, -0.066084072, 0.00411224086, -0.052153334, -0.0230224412, -0.0205028933, -0.0148822917, 0.0664010048, -0.0353399254, 0.00896058325, -0.0370276161, 0.00433198642, 0.00703231944, 0.0446652882, 0.00540511915, 0.0070884116, 0.00687262043, 0.0355492309, -0.0237096027, 0.00892738812, -0.0174259469, -0.0189707968, -0.0185533054, -0.0434698202, 0.0249076057, -0.0061862031, 0.0500153787, -0.0233756695, 0.022243714, 0.00375360949, -0.0341175869, -0.0515319705, -0.0368419513, -0.0715411529, -0.0425350331, 0.0568428524, 0.0294694174, 0.0601997674, 0.012903708, -0.0409387499, 0.0129674328, -0.0973280519, 0.0253901575, 0.00332091, -0.0438595824, 0.0156236086, 0.0347611755, 0.00976946764, -0.0179446023, -0.0526299402, 0.021516541, 0.0019031188, -0.0269328281, 0.0266715102, 0.0264121685, -0.0458507575, 0.00742622698, -0.00239242823, 0.00101631088, 0.0215476118, 0.00383561244, -0.0142397527, -0.0085424874, -0.0620511658, -0.0248422232, -0.0426566117, -0.0112887435, -0.0201841239, 0.000690475514, -0.0559922718, -0.0126789883, -0.0183034912, 0.0273448881, 0.0103349909, 0.0147193838, -0.0460172556, 0.0274905954, 0.0325787477, 0.00632856414, -0.0390700288, -0.00397625053, -0.00264929468, -0.0513475165, 0.0447640233, -0.0105459429, -0.0250227824, 0.107420661, 0.0493374392, 0.0271743797, -0.0581548028, 0.0659172535, -0.0567341708, 0.096310012, -0.0136204939, 0.00892539, 0.000444213278, -0.027534876, 0.00814599, 0.0484181643, -0.00172533677, 0.0396861956, -0.0123282084, -0.0223322678, 0.0117884474, 0.0236529931, -0.0928098112, 0.010599046, 0.0513251051, -0.0394491926, 0.0715423152, -0.000702124147, 0.0252006035, -0.0187341738, 0.0190776754, -0.00374436, 0.0242118519, 0.000415435614, 0.0250112396, -0.0622649, 0.00252862368, -0.0274931267, 0.0391479656, 0.0743277818, 0.0972099, -0.0616396628, 0.00200561038, -0.0558071248, 0.0217129104, -0.0134909321, 0.0102222189, 0.0610636771, 0.00289361388, 0.0140922582, -0.0253831036, -0.0203524046, 0.0109083308, -0.00374992541, -0.000384735147, -0.0503201596, -0.0329827666, 0.0136934193, 0.0464733131, -0.0360739492, -0.0295595676, -0.00903569628, -0.0639803484, 0.00282565528, 0.0342626, -7.62020752e-33, 0.0348942503, -0.0359011553, 0.0247418545, 0.00481113093, -0.0480940789, 0.0204219855, -0.0298511237, -0.0145599395, -0.0668907687, 0.00219926517, -0.0805463418, -0.00805195235, 0.0092622079, -0.00357259717, 0.0266291741, 0.0633839443, -0.0376900695, 0.0262610838, -0.00238141813, -0.0342469, 0.024246892, 0.0110699488, 0.0624549277, -0.0328984931, 0.0583810098, 0.00357276504, -0.016244024, -0.0458295718, 0.0472441129, 0.00207011984, 0.0191175025, -0.0320127457, 0.0225738958, -0.0552259646, 0.0166958049, -0.0610796548, -0.0919538215, -0.00636024307, -0.00634803157, 0.0279138889, 0.0216747113, 0.0034947542, 0.0425596908, -0.00329550682, -0.0315621234, 0.00323228654, 0.0611003824, 0.00199710461, -0.0870301127, -0.0540871099, 0.000970375841, -0.0131091615, 0.00164378132, 0.121977106, -0.0389591493, 0.0458402187, -0.0251122471, 0.0212387424, -0.0649295747, 0.00582447182, 0.00707374467, 0.0893055126, 0.0159334373, -0.0100898268, 0.0304512791, -0.00093530817, 0.023985086, 0.0190823935, 0.0197523236, -0.0207654797, -0.00747683551, 0.0139080603, -0.066028, 0.0985070094, 0.0137844067, -0.0940804183, -0.0491746664, 0.0425573625, 0.0581045821, -0.00544359349, 0.0117936283, -0.0209728722, -0.0314563923, -0.0466105305, 0.0473259389, 0.054147, 0.0107649788, -0.0359382778, 0.026368605, -0.0455537923, 0.0161353145, 0.0107065896, 0.0123144258, -0.0581069179, -0.0335470065, 0.0623139814, -0.0124190068, 0.0442875326, -0.0259920452, -0.0131240385, -0.0927172899, -0.0496345349, -0.0505607463, 0.0635729581, 0.00326133496, -0.00466958713, 0.0164645128, 0.0162077341, -0.038486477, -0.0229508467, 0.034288466, -0.0204868261, 0.0391127951, -0.0133459624, -0.0589093193, -0.00451791566, 0.0327330343, -0.0386987664, 0.0757958144, 0.0381116681, -0.00948850438, -0.0517759435, -0.044844836, -0.0000633628, 0.00875712372, 0.00309004728, -0.0462804697, 0.0450426228, 0.00800026581, -0.0119164707, -0.00346010807, -0.049270045, 3.29097475e-7, 0.0234622192, 0.0054900567, -0.0140785202, -0.0101590017, 0.0277022347, -0.0164527614, -0.0120923398, 0.0406606905, 0.0118274484, 0.0376342498, 0.0650030822, -0.0213564206, -0.0200665742, -0.00323811709, -0.0563624725, -0.0227815937, -0.0933469906, -0.0223109554, -0.0173683576, 0.0274538212, -0.0341707915, -0.0232605971, -0.0177894924, 0.00316540082, -0.00475196401, -0.0581303351, 0.00237789494, -0.0555701554, 0.0113115786, 0.0366914943, 0.0145752663, -0.0110980654, -0.00587131316, 0.0175981093, -0.0154663716, -0.0141303912, -0.00546650076, -0.000582889712, 0.0167552233, 0.0747121796, 0.0100347968, -0.0366787426, 0.0109702768, -0.0450229645, 0.0660410598, 0.0528041087, -0.0850212499, 0.0244038124, -0.0467818193, -0.0398042537, 0.012778651, -0.00283739157, 0.0135533772, 0.00978401, -0.0348206721, 0.0219416525, 0.00742749, 0.0121356435, -0.00928652193, -0.00523139816, 0.00130310119, -0.0798810869, -0.00906001683, -0.0293319337, 0.0363927968, 0.012738592, 0.0156276394, 3.37076472e-34, 0.0126905199, -0.0333545916, -0.0191886947, 0.00520856306, 0.0222699642, -0.0139833856, -0.055403892, -0.00884083379, -0.0148334522, -0.0063038012, -0.0276833083]

In [216]:
query_results1 = index.query(
    namespace="test",
    vector=test_vector,
    top_k=10,
    # include_values=True
    # include_metadata=True
)

query_results1

{'matches': [{'id': 'doc_chunk_4', 'score': 1.0, 'values': []},
             {'id': 'doc_chunk_6', 'score': 0.843322873, 'values': []},
             {'id': 'doc_chunk_2', 'score': 0.743641257, 'values': []},
             {'id': 'doc_chunk_5', 'score': 0.556426167, 'values': []},
             {'id': 'doc_chunk_8', 'score': 0.522498786, 'values': []},
             {'id': 'doc_chunk_1', 'score': 0.509563506, 'values': []},
             {'id': 'doc_chunk_0', 'score': 0.46634388, 'values': []},
             {'id': 'doc_chunk_3', 'score': 0.444756329, 'values': []},
             {'id': 'doc_chunk_10', 'score': 0.417294651, 'values': []},
             {'id': 'doc_chunk_7', 'score': 0.387660056, 'values': []}],
 'namespace': 'test',
 'usage': {'read_units': 5}}

In [208]:
# index.delete(delete_all=True, namespace="test")

{}

In [11]:
os.getcwd()

'c:\\vidit\\codes\\PROJECTS\\Tutor-app'

In [2]:
os.chdir("../")

In [11]:
import os
from pinecone import Pinecone, ServerlessSpec
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader
from llama_index.vector_stores.pinecone import PineconeVectorStore
from IPython.display import Markdown, display
from llama_parse import LlamaParse
import nest_asyncio

nest_asyncio.apply()

llama_api_key = os.getenv("LLAMA_API_KEY")
parser = LlamaParse(api_key=llama_api_key, result_type="markdown")    # Better than pypdf

file_extractor = {".pdf": parser,".csv":parser,".docx":parser,".txt":parser}
documents = SimpleDirectoryReader("./artifacts/csvs",file_extractor=file_extractor).load_data()  

print(f"Loaded {len(documents)} documents")

Started parsing the file under job_id 9bdb9191-1581-48eb-ba0e-42b493304fe8
Started parsing the file under job_id ef27e893-e5e0-4909-9095-913c766809a2
Loaded 2 documents


In [12]:
from llama_index.llms.ollama import Ollama
from llama_parse import LlamaParse
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader, PromptTemplate
from llama_index.core.embeddings import resolve_embed_model
from llama_index.core.tools import QueryEngineTool, ToolMetadata
from llama_index.core.agent import ReActAgent
from pydantic import BaseModel
from llama_index.core.output_parsers import PydanticOutputParser
from llama_index.core.query_pipeline import QueryPipeline
from prompts import context, code_parser_template
from code_reader import code_reader
from dotenv import load_dotenv
import os
import ast


vector_store = PineconeVectorStore(pinecone_index=pc_index_instance)
storage_context = StorageContext.from_defaults(vector_store=vector_store)
index = VectorStoreIndex.from_documents(
    data, 
    storage_context=storage_context,
    embed_model=embedding_model
    )

AttributeError: 'LlamaParse' object has no attribute 'parse'